In [1]:
from rwa_model import RWA
from utils import AddTask

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np

import matplotlib.pyplot as plt
from tensorboard_logger import configure, log_value

configure("training/run6")

In [2]:
num_inputs = 2
batch = 100
rwa = RWA(num_inputs, 250, 1, fwd_type="cumulative")

rwa.train()

criterion = nn.MSELoss()

print_steps = 10

current_lr = 1e-3
optimizer = optim.Adam(rwa.parameters(), lr=current_lr)

running_loss = 0.0
time_since_decay = 0
accumulated_loss = []

test = AddTask(100000, 10000, 100)

data_loader = DataLoader(test, batch_size=batch, shuffle=True, num_workers=4)

s, n, d, h, a_max = rwa.init_sndha(batch)

for epoch in range(5):
    

    for i, data in enumerate(data_loader, 0):

        inputs, labels = data
        inputs = Variable(inputs)
        labels = Variable(labels)

        rwa.zero_grad()
        outputs, s, n, d, h, a_max = rwa(inputs, s, n, d, h, a_max)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        s = Variable(s.data)
        n = Variable(n.data)
        d = Variable(d.data)
        h = Variable(h.data)
        a_max = Variable(a_max.data)

        running_loss += loss.data[0]
        accumulated_loss.append(loss.data[0])
        time_since_decay += 1

        if i % print_steps == print_steps-1:
            current_step = i + 1 + len(data_loader) * epoch
            log_value("Loss", running_loss/print_steps, step=current_step)
            log_value("LR", current_lr, step=current_step)
            log_value("Error", np.abs(outputs.data[0, 0] - labels.data[0, 0]), step=current_step)

#             if np.abs(np.mean(np.diff(accumulated_loss))) <= current_lr and time_since_decay >= 0.1 * len(data_loader):
#                 torch.save(rwa.state_dict(), "models/add.dat")
#                 current_lr = max([current_lr * 1e-1, 1e-8])
#                 print("lr decayed to: ", current_lr)
#                 optimizer = optim.Adam(rwa.parameters(), lr=current_lr)
#                 accumulated_loss.clear()
#                 time_since_decay = 0
                
            running_loss = 0.0

torch.save(rwa.state_dict(), "models/rwa_add.dat")
print("Finished Training")

AttributeError: type object 'object' has no attribute '__getattr__'